In [3]:
from huggingface_hub import login
login()

In [ ]:
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments
)
from peft import LoraConfig, get_peft_model

# ----------------------------
# Load UFAL dataset
# ----------------------------
eng_file = "ufal_dataset/ufal-nla-v1.eng"
da_file  = "ufal_dataset/ufal-nla-v1.apc"

with open(eng_file, encoding="utf-8") as f:
    eng_sents = [l.strip() for l in f if l.strip()]

with open(da_file, encoding="utf-8") as f:
    da_sents = [l.strip() for l in f if l.strip()]

assert len(eng_sents) == len(da_sents)

prompts, completions = [], []

for eng, da in zip(eng_sents, da_sents):
    prompts.append(
        "Translate the following English sentence into Syrian Arabic:\n"
        + eng + "\n"
    )
    completions.append(" " + da)

    prompts.append(
        "Translate the following Syrian Arabic sentence into English:\n"
        + da + "\n"
    )
    completions.append(" " + eng)

dataset = Dataset.from_dict({
    "prompt": prompts,
    "completion": completions
})

dataset = dataset.train_test_split(test_size=0.05, seed=42)

# ----------------------------
# Tokenizer
# ----------------------------
model_name = "inceptionai/Jais-2-8B-Chat"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast=False,
    trust_remote_code=True
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

MAX_LEN = 512

# ----------------------------
# Tokenization with prompt masking
# ----------------------------
def tokenize_fn(batch):
    input_ids, labels, attention_masks = [], [], []

    for p, c in zip(batch["prompt"], batch["completion"]):
        full_text = p + c

        tok_full = tokenizer(
            full_text,
            truncation=True,
            max_length=MAX_LEN
        )

        tok_prompt = tokenizer(
            p,
            truncation=True,
            max_length=MAX_LEN
        )

        ids = tok_full["input_ids"]
        prompt_len = len(tok_prompt["input_ids"])

        lbls = [-100] * prompt_len + ids[prompt_len:]
        lbls = lbls[:MAX_LEN]

        input_ids.append(ids)
        labels.append(lbls)
        attention_masks.append([1] * len(ids))

    return {
        "input_ids": input_ids,
        "labels": labels,
        "attention_mask": attention_masks
    }

tokenized = dataset.map(
    tokenize_fn,
    batched=True,
    remove_columns=dataset["train"].column_names
)

# ----------------------------
# Model
# ----------------------------
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.bfloat16,  # USE BF16
    trust_remote_code=True
)

model.gradient_checkpointing_enable()
model.config.use_cache = False
model.config.pad_token_id = tokenizer.pad_token_id

# ----------------------------
# LoRA
# ----------------------------
lora_config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True,
    label_pad_token_id=-100
)

# ----------------------------
# Training arguments (FAST + SAFE)
# ----------------------------
training_args = TrainingArguments(
    output_dir="./JAIS_SYR_LORA",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,     # effective batch = 16
    learning_rate=3e-5,
    warmup_ratio=0.03,
    max_grad_norm=1.0,
    num_train_epochs=2,               # enough for 120k
    bf16=True,
    fp16=False,
    logging_steps=500,
    save_steps=2000,
    save_total_limit=3,
    report_to="none"
)

# ----------------------------
# Trainer
# ----------------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    data_collator=data_collator
)

trainer.train()


Map:   0%|          | 0/229140 [00:00<?, ? examples/s]

Map:   0%|          | 0/12060 [00:00<?, ? examples/s]

`torch_dtype` is deprecated! Use `dtype` instead!


Loading weights:   0%|          | 0/516 [00:00<?, ?it/s]

warmup_ratio is deprecated and will be removed in v5.2. Use `warmup_steps` instead.


trainable params: 13,631,488 || all params: 8,104,032,768 || trainable%: 0.1682


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig
import torch

# Paths
base_model_name = "inceptionai/Jais-2-8B-Chat"
lora_model_path = "./JAIS_SYR_LORA/checkpoint-18000"
merged_model_path = "./JAIS_SYR_LOR_Amerged_UH"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    torch_dtype=torch.float16
)

# Load LoRA adapter
model = PeftModel.from_pretrained(model, lora_model_path)

# Merge LoRA weights into the base model
model = model.merge_and_unload()  # This merges LoRA into base

# Save merged model
model.save_pretrained(merged_model_path)
tokenizer.save_pretrained(merged_model_path)

print(f"Merged model saved to {merged_model_path}")



`torch_dtype` is deprecated! Use `dtype` instead!


Loading weights:   0%|          | 0/516 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Merged model saved to ./JAIS_SYR_LOR_Amerged_UH
